In [ ]:
# Installer les packages 
# On a utilisé Lightning Ai pour entrainer notre FRCNN puisqu'il met à notre disposition un gpu puissant.

%pip install roboflow 
%pip install pycocotools
%pip install -r /teamspace/studios/this_studio/Object-Detection/FRCNN/requirements.txt

In [ ]:
# Télecharger notre dataset 
from roboflow import Roboflow

rf = Roboflow(api_key="KhV6y3SVx0M66EFe23Vv")
project = rf.workspace("bootit").project("yolov5-peoples-cars")
version = project.version(8)
dataset = version.download("voc")

In [ ]:
# Lancer le script d'augmentations de données (augmentation géométriques et brumes)
!python /teamspace/studios/this_studio/Object-Detection/FRCNN/aug_frcnn.py -c /teamspace/studios/this_studio/Object-Detection/FRCNN/data_configs/augmentation.yaml

In [ ]:
# Entrainer notre modèle pour 30 epochs pour qu'on puisse l'utiliser dans la génération des attaques adversariales
!python /teamspace/studios/this_studio/Object-Detection/FRCNN/train.py --model fasterrcnn_resnet50_fpn_v2 --config /teamspace/studios/this_studio/Object-Detection/FRCNN/data_configs/ppe.yaml --batch-size 8 --epochs 30 --project-name frcnn --no-mosaic

In [ ]:
# Lancer le script des attaques adversariales

!python /kaggle/working/Object-Detection/yolov9/fgsm_yolov9.py --config /kaggle/working/Object-Detection/yolov9/data_configs/fgsm.yaml

In [ ]:
# Réentainer encore notre modèle sur la base de donnée finale en utilisant le modèle déja obtenu dès la première tentative d'entrainement
!python /teamspace/studios/this_studio/Object-Detection/FRCNN/train.py --model fasterrcnn_resnet50_fpn_v2 --weights /teamspace/studios/this_studio/outputs/training/frcnn/best_model.pth --config /teamspace/studios/this_studio/Object-Detection/FRCNN/data_configs/ppe.yaml --batch-size 8 --epochs 100 --project-name frcnn_100_epochs --no-mosaic